# Mapping 

Transportation is about getting from place A to place B.  Therefore, most transportation data has a spatial component to it.  It is nice to be able to put these data on a map and see what is going on.  It is even better if we can put it on a map and interact with the data.  It would be even cooler if we could put our interactive map on a website to show it off!

To do this, we are going to use a package called folium.  You can find the documentation here: 

https://folium.readthedocs.io/en/latest/

And access it on github here: 

https://github.com/python-visualization/folium


### Credits

This lesson draws from the folium quickstart notebook, and from Vik Paruchuri DataQuest lesson: 

https://www.dataquest.io/blog/python-data-visualization-libraries/

### A side note on static mapping

Sometimes you may want to create a static map instead of an interactive map.  Interactive maps are nice for exploring your data, but static maps work well for an image that you can insert into a paper.  If you want to create static maps, then basemap is a good tool.  Here is a nice lesson focused on mapping earthquake activity: 

http://introtopython.org/visualization_earthquakes.html



### OK, back to interactive mapping, because that's fun...

It turns out that folium doesn't do much itself.  It is just a wrapper around something called leafletjs.  You can read more about that here:

http://leafletjs.com/index.html

Leaflet is a library in the JavaScript language.  JavaScript is the language used for most web applications.  We could do the same thing using JavaScript and leaflet directly, but then we would have to learn the syntax for another language.  That might not be too hard, but to keep it simple, we'll stick to the python wrapper for now.  It is good to be aware of, though, because if you want more options than folium allows, you can go directly to leaflet.  

What makes this possible is the fact that leaflet has a well-defined API.  That means that we can pass data back and forth, even from a different language.  


In [1]:
import folium

In [2]:
m = folium.Map(location=[41.8781,-87.6298])

In [3]:
m

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.formula.api as smf
import geopandas as gp
from scipy import stats, integrate
import matplotlib.pyplot as plt

# this allows plots to appear directly in the notebook
%matplotlib inline

In [5]:
# These files use \N as a missing value indicator.  When reading the CSVs, we will tell
# it to use that value as missing or NA.  The double backslash is required because
# otherwise it will interpret \N as a carriage return.
trips = pd.read_csv("/Users\Daniel\Desktop\CE 599\ce_599_new\METRA-Rail-Delay-Study/trip.csv", header=None, na_values='\\N')
trips.columns = ["time", "time_formated", "id", "route_id", "vehicle_id", "vehicle_label", "delay", "lat", "lon", "general_weather", "temp", "temp_min", "temp_max", "visibility", "wind_speed"]
trips["time_formated"]=pd.to_datetime(trips["time_formated"])

In [6]:
trips['time_new']=pd.to_datetime(trips['time_formated'],format='%H:%M:%S').dt.time
trips

,time,time_formated,id,route_id,vehicle_id,vehicle_label,delay,lat,lon,general_weather,temp,temp_min,temp_max,visibility,wind_speed,time_new
0,1554009576,2019-04-24 00:19:36,UP-N_UN835_V6_B,UP-N,8413,835,300,42.346638,-87.829590,"[{'id': 802, 'main': 'Clouds', 'description': ...",271.76,270.15,273.15,16093.0,3.6,00:19:36
1,1554009576,2019-04-24 00:19:36,BNSF_BN1328_V6_B,BNSF,8584,1328,300,41.845730,-87.738174,"[{'id': 803, 'main': 'Clouds', 'description': ...",272.22,271.48,273.15,16093.0,6.7,00:19:36
2,1554009608,2019-04-24 00:20:08,UP-N_UN835_V6_B,UP-N,8413,835,300,42.353279,-87.828880,"[{'id': 802, 'main': 'Clouds', 'description': ...",271.76,270.15,273.15,16093.0,3.6,00:20:08
3,1554009608,2019-04-24 00:20:08,BNSF_BN1328_V6_B,BNSF,8584,1328,300,41.847168,-87.732002,"[{'id': 803, 'main': 'Clouds', 'description': ...",272.23,271.48,273.15,16093.0,6.7,00:20:08
4,1554009639,2019-04-24 00:20:39,UP-N_UN835_V6_B,UP-N,8413,835,300,42.356483,-87.828545,"[{'id': 803, 'main': 'Clouds', 'description': ...",271.75,270.15,273.15,16093.0,2.6,00:20:39
5,1554009639,2019-04-24 00:20:39,BNSF_BN1328_V6_B,BNSF,8584,1328,240,41.848778,-87.724808,"[{'id': 803, 'main': 'Clouds', 'description': ...",272.23,271.48,273.15,16093.0,6.7,00:20:39
6,1554009670,2019-04-24 00:21:10,UP-N_UN835_V6_B,UP-N,8413,835,300,42.357857,-87.828392,"[{'id': 803, 'main': 'Clouds', 'description': ...",271.75,270.15,273.15,16093.0,2.6,00:21:10
7,1554009670,2019-04-24 00:21:10,BNSF_BN1328_V6_B,BNSF,8584,1328,240,41.850452,-87.717232,"[{'id': 803, 'main': 'Clouds', 'description': ...",272.23,271.48,273.15,16093.0,6.7,00:21:10
8,1554009702,2019-04-24 00:21:42,UP-N_UN835_V6_B,UP-N,8413,835,300,42.357857,-87.828392,"[{'id': 803, 'main': 'Clouds', 'description': ...",271.75,270.15,273.15,16093.0,2.6,00:21:42
9,1554009702,2019-04-24 00:21:42,BNSF_BN1328_V6_B,BNSF,8584,1328,180,41.852768,-87.706955,"[{'id': 803, 'main': 'Clouds', 'description': ...",272.22,271.48,273.15,16093.0,6.7,00:21:42


In [7]:
# let's peek at what we have
trips.head()

,time,time_formated,id,route_id,vehicle_id,vehicle_label,delay,lat,lon,general_weather,temp,temp_min,temp_max,visibility,wind_speed,time_new
0,1554009576,2019-04-24 00:19:36,UP-N_UN835_V6_B,UP-N,8413,835,300,42.346638,-87.829590,"[{'id': 802, 'main': 'Clouds', 'description': ...",271.76,270.15,273.15,16093.0,3.6,00:19:36
1,1554009576,2019-04-24 00:19:36,BNSF_BN1328_V6_B,BNSF,8584,1328,300,41.845730,-87.738174,"[{'id': 803, 'main': 'Clouds', 'description': ...",272.22,271.48,273.15,16093.0,6.7,00:19:36
2,1554009608,2019-04-24 00:20:08,UP-N_UN835_V6_B,UP-N,8413,835,300,42.353279,-87.828880,"[{'id': 802, 'main': 'Clouds', 'description': ...",271.76,270.15,273.15,16093.0,3.6,00:20:08
3,1554009608,2019-04-24 00:20:08,BNSF_BN1328_V6_B,BNSF,8584,1328,300,41.847168,-87.732002,"[{'id': 803, 'main': 'Clouds', 'description': ...",272.23,271.48,273.15,16093.0,6.7,00:20:08
4,1554009639,2019-04-24 00:20:39,UP-N_UN835_V6_B,UP-N,8413,835,300,42.356483,-87.828545,"[{'id': 803, 'main': 'Clouds', 'description': ...",271.75,270.15,273.15,16093.0,2.6,00:20:39


In [8]:
#trips = trips[:1000]

In [9]:
for name, row in trips.iterrows():
    marker = folium.CircleMarker([row['lat'], row['lon']], radius=2, popup=str(row['delay']))
    marker.add_to(m)

In [10]:
m

In [ ]:
for name, row in trips.iterrows():
    
    if row["id"] != "Rogers Park":
        marker = folium.CircleMarker([row["lat"], row["lon"]], 
                                     radius=5,
                                     color='Chartreuse',
                                     fill_color='Chartreuse', 
                                     popup=row['id'])
        marker.add_to(m)
        
         
m.save('m.html')

In [ ]:
# create a basic map, centered on Chicago Metra Rail
chicago_air = folium.Map(
    location=[42.328,-87.837],
    tiles='Stamen Toner',
    zoom_start=4
)

In [ ]:
# Define some empty sets
delay_set = set()
time_set = set()

# Make sure we don't add duplicates, especially for the origins
for name, row in trips.iterrows():
    
    if row['delay'] not in delay_set: 
        popup_string = str(row['delay']) + ' (' + str(row['delay']) + ')'
        marker = folium.CircleMarker([row["lat"], row["lon"]], 
                                     color='Brown',
                                     fill_color='Brown', 
                                     radius=5, popup=popup_string)
        marker.add_to(m)
        delay_set.add(row['delay'])
    
    if row['delay'] not in time_set: 
        popup_string = str(row['time']) + '(' + str(row['delay']) + ')'
        marker = folium.CircleMarker([row["lat"], row["lon"]], 
                                     color='MidnightBlue',
                                     fill_color='MidnightBlue', 
                                     radius=5, popup=popup_string)
        marker.add_to(m)
        time_set.add(row['delay'])

m.save('m.html')
  

In [ ]:
trips.delay.value_counts()

In [ ]:
trips_new = trips[trips['delay']>0]
trips_new

In [ ]:
def delaycolors(counter):
    if counter['delay'] == 60:
        return 'green'
    elif counter['delay'] == 120:
        return 'blue'
    elif counter['delay'] == 180:
        return 'red'
    elif counter['delay'] == 240:
        return 'white'
    elif counter['delay'] == 300:
        return 'pink'
    elif counter['delay'] == 360:
        return 'purple'
    elif counter['delay'] == 420:
        return 'black'
    elif counter['delay'] == 480:
        return 'orange'
    elif counter['delay'] == 540:
        return 'beige'
    else:
        return 'darkblue'
trips_new["color"] = trips_new.apply(delaycolors, axis=1)
trips_new.head()

In [ ]:
# Define some empty sets
delay_set = set()
time_set = set()

# Make sure we don't add duplicates, especially for the origins
for name, row in trips_new.iterrows():
    
    
    popup_string = str(row['delay']) + ' (' + str(row['delay']) + ')'
    marker = folium.CircleMarker([row["lat"], row["lon"]], 
                                 radius=5, color=row['color'], fill_color=row['color'], popup=popup_string)
    marker.add_to(m)
        
        

m.save('m.html')

In [ ]:
trips_new

In [ ]:
sns.distplot(trips['delay'])

In [ ]:
sns.distplot(trips['visibility'])

In [ ]:
sns.jointplot(x="delay", y="temp", data=trips, size=8)

In [ ]:
sns.regplot(x="delay", y="temp", data=trips)

In [ ]:
mod = smf.ols(formula='delay ~ temp', data=trips)
res = mod.fit()
print(res.summary())

In [ ]:
trips.general_weather.value_counts()

In [ ]:
mod = smf.ols(formula='delay ~ route_id', data=trips)
res = mod.fit()
print(res.summary())

In [ ]:
str("route_id")

In [ ]:
trips.route_id.value_counts()

In [ ]:
ax = sns.violinplot(x="route_id", y="delay", data=trips_new)

In [ ]:
sns.violinplot( x=trips_new["route_id"], y=trips_new["delay"], linewidth=2)

In [ ]:
sns.violinplot( x=trips_new["route_id"], y=trips_new["delay"], width=1.5)

In [ ]:
ax = sns.violinplot(x="route_id", y="delay", data=trips_new)
sns.violinplot( x=trips_new["route_id"], y=trips_new["delay"], linewidth=2)
sns.violinplot( x=trips_new["route_id"], y=trips_new["delay"], width=1.8)
sns.violinplot(x='route_id', y='delay', data=trips_new, order=[ "UP-N", "MD-W", "UP-W", "RI", "UP-NW", "BNSF", "MD-N", "ME"])

plt.title('Route ID versus Delay')
plt.xlabel('Route ID')
plt.ylabel('Delay')

In [ ]:
sns.violinplot(x="time_formated", y="delay", data=trips_new)

In [ ]:
plt.title('Time Formated versus Delay')
plt.xlabel('Time Formated')
plt.ylabel('Delay')
 
# Limits for the Y axis
plt.ylim(0,600)
plt.xlim(0,60)
 
# Create names
plt.xticks()
 
# Show graphic
plt.show()


In [ ]:
sns.violinplot(x="vehicle_id", y="delay", data=trips_new)

In [ ]:
ax = sns.violinplot(x="vehicle_id", y="delay", data=trips_new)
sns.violinplot( x=trips_new["vehicle_id"], y=trips_new["delay"], linewidth=2)
sns.violinplot( x=trips_new["vehicle_id"], y=trips_new["delay"], width=1.8)


plt.title('Vehicle ID versus Delay')
plt.xlabel('Vehicle ID')
plt.ylabel('Delay')

In [ ]:
trips.vehicle_id.value_counts()

In [ ]:
trips_new['delay'].groupby([trips_new['route_id'],trips_new['vehicle_id']]).describe()

In [ ]:
trips_new['delay'].groupby(trips_new['vehicle_id']).describe()

In [ ]:
mod = smf.ols(formula='delay ~ C(route_id)', data=trips)
res = mod.fit()
print(res.summary())